# Summary of Code
Converting IMU data (gyroscope and accelerometer data), I am trying to make a simple digital twin of the spine which can be used as a posture correction device.

# Next Steps
- quaternions (needs to be done soon). Need to understand the theory and try to properly integrate this. However, for the code I will be using `scipy.spatial.transform.Rotation`rather than writing it all out myself for the sake of efficiency.

Here are some videos for this:
- **https://www.youtube.com/watch?v=hZrBGiWkUyk&t=504s**: *this actually shows a python integration, but quite lax on the theory behind it all*
- https://youtu.be/bKd2lPjl92c?si=yyqcKvoMNcVktlbg
- https://youtu.be/zjMuIxRvygQ?si=b8NkCf_LGxgAJK3Y

---------------

- basic export of data (.csv) and a simple way of scaling (it could be incorporated into a function and then be transferred into an sqlite database)
    - This will later be used as the basis for ML training data

--------------

- improved interpretation of curvature and deviations using clustering ML techniques to help with this project

## Spline Interpolation
*A pretty big note for this is that the constant of 250 (being how many indices are in the list, which can change to make the reading of results more accurate)*


- Get the hospot points plotting
- Segment the spine into positions wrt the inputted linear distances
- Test the functionality of code and debug
- Send off the position, type and intensity of deviance in serial (e.g. serial.bwrite'{3},{STATIC},{HIGH}')
- Debug and iterate with that (and record results with sensors)

--------

- Reducing the accumulation of noise:
    - looking into methods of fixing the rotation issue (like with the issues in the forward kinematics algorithm) so that the resulting curves look like curves
        - look into quaternions
        - look into other approaches to this issue
- Making the curve seem more spine-like:
    - looking into biomechanics to inform my application of B-splines

--------

- Useability:
    - Try to focus on the interface for this project (**in December, when every other part of the project is completely finished off**) so that I can prepare properly for the exhibition, where everyone can use the device (having gone through the calibration phase) and give feedback on the design, wearability and all that


In [ ]:
import serial.tools.list_ports
import string
import serial
import matplotlib.pyplot as plt
from  mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import time
from collections import deque
import numpy as np
from scipy.interpolate import CubicSpline
import scipy
import sys # for debugging specifically

### Serial Reading & Setup

In [ ]:
BAUDRATE = 115200
try:
    # setup for port communication
    ports = serial.tools.list_ports.comports()
    serialInst = serial.Serial()
    portList = [str(i) for i in ports]
    print(portList)

    com = input("Select COM PORT for Arduino: ")

    for i in range(len(portList)):
        if portList[i].startswith("COM" + str(com)):
            SERIAL_PORT = "COM" + str(com)
            print(SERIAL_PORT)

    serialInst.baudrate = BAUDRATE
    serialInst.port = SERIAL_PORT
    serialInst.open()
    print(f"Connected to {SERIAL_PORT} at {BAUDRATE} baud.")

    '''
    initial setup: 'begin program'
    '''

    while True:
        line = serialInst.readline().decode('utf-8') #.strip()
        if line: # if there is data in the readline
            print(f"Arduino: {line}")
        if "Available channels:" in line:
            # extract all after the colon
            channels_part = line.split(":")[-1].strip()
            # parse the comma separated list into on stringed list
            IMU_ID_LIST = [id.strip() for id in channels_part.split(",") if id.strip()]
        if "Number of sensors: " in line:
           ID_NUM = int(line.strip(":")[-3]) # the number of read sensors, last instance is "\n" and so index = -3 is the appropriate index
           #print(ID_NUM)
           IMU_DEQUES = [deque(maxlen=50) for i in range(ID_NUM)]
        if "Waiting for 'begin program' command" in line:
            break

    # get the linear distances for the forward kinematics calculation
    print("INSTRUCTIONS:\nInput the linear distances between your sensors in metres.\nMeasure from lowest to highest.\nI would recommend using a high resolution ruler to reduce drift.\n")
    linear_distances = []
    for i in range(ID_NUM - 1):
        value = float(input(f"{i + 1}: "))
        linear_distances.append(value)

    print("Sending 'begin' command to Arduino")
    serialInst.write(b'begin program') # sent in bytes rather than high level strings
                                       # since it is sent to the compiler

    time.sleep(2)

### Functions & Algorithms
I am here using a forward kinematics algorithm that inputs the matrix of accelerometer data (raw data?) and outputs a kinematic chain that approximates the scaled relationship between the sensors with the inputted linear distances.

**Segmentation Algorithm**: The output of this is such that $[n_0, n_1, ..., n_i]$, where $0 < n < 1$ and $i$ is the length of the list. Using this format, $n_i \equiv 1$ since it is a cumulative algorithm that indicates the percentiles in which these values should occur.

Using this segmentation, I can then send data to the nearest haptic motor if a deviation has occurred.


In [ ]:
    def forward_kinematics(matrix, linear_distances):
        '''
        Computes positions using the IMU data and distances, assuming that the base IMU is at the origin.
        Returns: list of 3D positions

        For later improvements, I will use quaternions to handle the tilt as done in the CHARM Lab device.
        '''

        if not matrix:
            return []

        positions = [np.array([0,0,0])] # centre at the origin, the position vectors are arbitrary as long as they are scaled versions of reality
        t_value_distance = [0]
        cumulative_distance = 0

        for i in range(1, len(matrix)):
            # direction_vector is the difference in the vector direction of the sandwiched positions
            direction_vector_i = np.array(matrix[i])
            direction_vector_j = np.array(matrix[i-1])

            direction_vector = direction_vector_i - direction_vector_j # relative direction
            norm = np.linalg.norm(direction_vector)
            if norm > 1e-10: # error handling incase it produces a small value (and handles for floating point error)
                direction_vector = direction_vector / norm
            else: # error handling
                if i==1:
                    direction_vector = np.array([0,0,1]) # simply translate the vector directly above it
                else:
                    # continue in the same direction as the previous linkage ()
                    prev_pos = positions[-1] - positions[-2]
                    direction_vector = prev_pos / np.linalg.norm(prev_pos)

            distance = linear_distances[i-1]
            link_vector = direction_vector * distance
            new_pos = positions[-1] + link_vector # build from previous position
            positions.append(new_pos)

            cumulative_distance += distance
            t_value_distance.append(cumulative_distance)

        return positions, t_value_distance

    def segmentation_algorithm(linear_distances_list):
        array = np.array(linear_distances_list)
        return np.cumsum(array) / np.sum(array)

    def interquartile_mean_calculation(poor_posture_list_input):
        list_input = np.array(poor_posture_list_input)
        n = len(list_input)

        if n == 0:
            return np.nan # an empty list
        if n < 4: # not enough elements for any meaningful IQR
            return np.mean(list_input)

        list_input.sort()
        n_upper = int(n * 0.75)
        n_lower = int(n * 0.25)

        interquartile_list = list_input[n_lower:n_upper]

        if interquartile_list.size == 0: # incase an unexpected error occurred
            return np.mean(list_input)

        interquartile_mean = np.mean(interquartile_list)
        return interquartile_mean

### Databases & .csv Files
It will be stored in a SQLite database

### Data Plotting

The previous runs of this had a monolithic `animate(i)` function which made this impossible to debug.

Therefore:
- develop a class based structure
- test this code in a separate file with pseudo data
- there is an error with the runtime of this with 5 sensors

All of these makes the implementation of ML (like `k-mean clustering` for the calibration phase) much harder to implement.

There should also be ways of quantifying the improvements of this code but I am not sure currently.

In [ ]:
'''
I do not know how to implement the state variables across both classes in any meaningful way; this will be a problem that I will solve later.
'''

class Spine:
    def __init__(self, raw_data, num_sensors):
        '''
        This holds the raw information of the digital twin except for any analysis.
        The output of this is the function of the spine at a given point (and the list of its positions for previous timesteps) but nothing else.
        Here there will be the storage into the database (SQLite) but I need to understand where this is actually stored and how to restore it after different runs of this code.
        '''

        self.raw_data = raw_data
        self.num_sensors = num_sensors
        self.poor_posture_start_time = None

    def data_cleaning(self):
        '''
        Returns cleaned IMU data through whatever filter or combination of them to make sure that these vectors actually display tilt (and not something else)
        '''
        return self.clean_data

    def interpolated_spline(self):
        '''
        This should also update the deque list (and how it is stored within the code)
        Also have qualities such as 'self.position_vectors' and a specific understanding of how many vectors to have and where they all go handled nicely
        '''
        return self.interpolated_spline # just returns the spline with no further data

    def database_storage(self):
        '''
        Call the database here and upload the raw data of the spine here and see the flaws in my recreation.
        Raw data is the only important aspect here so this should be quite a simple endeavour.

        The only issue here is the rate at which this function is called since there is now a real issue of threading and the rate of calling a function exceeding the rate it actually runs.
        How do I overcome this?
        '''

For the function `calibration_dataset(self)`, I am not sure whether this data should be raw or the already filtered curvatures from the `Spine` class. The filtered ones may be better since I could just improve the previous algorithm (using quaternions and filters to reduce the drift from the accelerometers) which would just save time.

I also want to display this output as a nodal graph (and cluster the data in different colours for each of the IMU segments) to try to visualise what this intuitively means.

For the function `calibrated_curvature(self)`, I am struggling to think of an appropriate output from this function given that I would have calculated the statistical distribution from the initial dataset but now what should I do with it. I want a method that is not computationally expensive (I don't want to calculate the range from which the data could fall into for each step). This will be a problem to solve, but I do not know when.

In [ ]:
class CurvatureAnalysis:
    def __init__(self):
        self.baseline_curvature = None
        self.calibration_duration = 30
        self.calibration_dataset = None

    def calibration_dataset(self, spline):
        '''
        First creates the database called `sql-database-{random_seed}` # the random seed is such that I can have different instances of databses when I have different tests of this
        This collects all the curvatures from the sensors during the calibration period into a dataset in which k-means clustering can take place.
        '''
        return self.calibration_dataset

    def calibrated_curvature(self, spline):
        '''
        Using the dataset, k-means clustering will take place such that a measure of deviation from this distribution can be assertained.
        My intention would be further for this data to be presented (not as a digital twin) to see whether more information can be ascertained.
        What would be returned from this? The ranges of data this can fall between to not be an anomaly?
        '''
    def curvature_dataset_update(self, spline, tag):
        # continue updating the same dataset from the calibration dataset but tag the data as `running` (or just not `calibrating` or deviating)
        # these tags will inform the database writing

    def curvature_deviance(self, spline):
        '''
        if Spine.interpolated_spline(spline) has not deviated from the above dataset:
            store as `not deviated` (have a better name for this)
            return None
        else:
            find where it has deviated (along the spline)
            store this data (how could this graphically be represented) - send to a different function to store the data in the sqlite database
            store as `deviated`
            try to isolate the indices where this has occurred along the spline
            call function that finds which IMU this should go to
            return [red-indices-of-deviance], [haptic-motor-sensors-to-ignite]
        '''


Need to have a separate database class since the timings for updates for this will be different (and I also want the updates to the datbase to be at a higher frequency than the visualisation plot of the animate function).

The file format is something similar to:
``` python
data = {
    "timestep": timestep,
    "tag": calibrating
    "IMU-raw-data": IMU-raw-data # (1 x 3) matrix
}
```
I would like to plot a graph of this (the points being the data points) and then having the timestep being the lower axis. This will help with the visualisation of this (befor the animation file for this).


In [ ]:
    class DatabaseLogger():
        def __init__(self):
            self.dbname = dbname



Below is the pseudocode for this, where I upload everything to a database nice and good to test the storage of this. Is this necessary?


1. I will have distinct functions (not classes) that will help with the creation of the spline object in real time without a class. These functions will:

    - clean the IMU data

    - produce an appropriate interpolation of this data (that will be outputted in similar formats such that it is modular, whether it be a vector of length n that is consistent regardless of how I improve the functions with increasingly complex quaternion math or madgwick filtering)

2. Now there will be a class that handles the stored interpreted data which needs the states of timings for the the different points( poor_posture_time, `calibration_duration`) as well as storing the local state of what the calibration function (or how to cluster the data points during this calibration time, I am thinking of k-means clustering and then defining a standard deviation for this (if there is a sharp change in curvature for a short period time that isn't noise) as well as a function that alterts the user if there has been posture that is prolonged within this frame (i.e. just sitting static in front of a screen for a while without any movement))

3. This then runs within the animate function but has been sectioned off to the different classes and functions and also towards the visualisation of the code

### Plot Setup

In [ ]:
    fig = plt.figure(figsize=(15,9))
    ax = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)

    ## set the points for the animate(i) function
    ## these variables are what are updated (and saves clearing the plot each time)
    scatter = ax.scatter([], [], [], s=50)
    line, = ax.plot([], [], [])
    scatter_hotspot = ax.scatter([], [], [], color='red', s=50)

    ax.set_title("IMU Positions")
    ax.set_xlabel("X (m)")
    ax.set_ylabel("Y (m)")
    ax.set_zlabel("Z (m)")

Below is the rewritten function in its entirety, where each section is run and the class-based system allows debugging to occur at a much more efficient rate.

In [ ]:
    def animate(i):
        try:
            '''
            1. Input and clean up the data from the IMU & sort out the channels of this so that the deques are running correctly
            2. Make the interpolated spline & curvature analysis instance
            3. Instructions that a calibration phase will take place and to get the user in a good position
            4. Calibration phase (with all the data at this point being inputted into the sql-database)
            5. The code running as normal; if a deviation (how is this defined) then the data is sent through the compiler otherwise just...
            6. Render the plot of:
                a. the interpolated spline function (the 'digital-twin')
                b. the k-means cluster (or way of approaching this problem)
            '''
        except Exception as e:
            print(f'Serial line error: {e}')
            return # plotting stuff

In [ ]:

    baseline_curvature = None
    is_calibrating = True
    calibration_start_time = time.time()
    calibration_duration = 3 # seconds; increase this time for actual deployment. It will be low for testing.
    calibration_curvature_array = [] # use np.array or something similar
    calibration_standard_deviation = [] # standard deviation of the points on the thing

    poor_posture_list = [] # contains the arrays of all the deviated postures
    poor_posture_start_time = None # storing the time.time() when poor posture starts
    threshold_time = 0.1 # seconds; it will increase again after testing

    plot_radius = sum(linear_distances) * 1.125 # a scaled form of the plot so that I can see the results more clearly

    linear_distance_percentiles = segmentation_algorithm(linear_distances) # positions of the linear distance percentiles
    print(f"\nCALIBRATION PHASE.\nDURATION: {calibration_duration}\n\n")
    def animate(i):
        # making these values global in this function will reduce the chance of errors
        global baseline_curvature, is_calibrating, calibration_start_time, calibration_curvature_array
        global curvature_standard_deviation, poor_posture_list, poor_posture_start_time

        try:
            """DATA ACQUISITION"""
            IMU_FULL_CHANNEL_DATA = serialInst.readline().decode('utf-8').strip().split(",")
            IMU_ID = int(IMU_FULL_CHANNEL_DATA[-1]) # the ID prescribed by the sensor number
            IMU_DATA = [float(acc) for acc in IMU_FULL_CHANNEL_DATA[:3]] # ONLY APPENDING ACCELERATION, TO CHANGE WHEN DOING KALMAN FILTERING
            IMU_DATA_NORM = IMU_DATA/np.linalg.norm(IMU_DATA)

            try:
                target_IMU_index = IMU_ID_LIST.index(str(IMU_ID)) # str(IMU_ID) since IMU_ID_LIST is a string
                IMU_DEQUES[target_IMU_index].append(IMU_DATA_NORM)

            except ValueError:
                print(f"Warning: Recieved data from unknown data channel @ ID:{IMU_ID}")

            if all(IMU_DEQUES):# when there is data in all of the deques i.e. all the sensors are outputting data
                IMU_NORMALISED_MATRIX = [dque[-1] for dque in IMU_DEQUES]
                positions, t_values = forward_kinematics(IMU_NORMALISED_MATRIX)
                IMU_POSITIONS = np.array(positions)

                x = IMU_POSITIONS[:, 0]
                y = IMU_POSITIONS[:, 1]
                z = IMU_POSITIONS[:, 2]

                # below is an example of a global cubic spline which will be easier to compute.
                # I will experiment later with other forms of parametric equations (e.g. constrained b-splines) but the simplicity of this will help with better optimisation
                xc = CubicSpline(t_values, x)
                yc = CubicSpline(t_values, y)
                zc = CubicSpline(t_values, z)

                plot_t = np.linspace(min(t_values), max(t_values), 250) # t_values to for a smooth plot

                curvature_list = [] # the list for values of $\kappa$ that updates for each instance of the spline curve
                for i in range(len(plot_t)):
                    r = (xc(plot_t[i], 0), yc(plot_t[i], 0), zc(plot_t[i], 0))
                    r_prime = (xc(plot_t[i], 1), yc(plot_t[i], 1), zc(plot_t[i], 1))
                    r_double_prime = (xc(plot_t[i], 2), yc(plot_t[i], 2), zc(plot_t[i], 2))

                    kappa = (np.linalg.norm(np.cross(r_prime, r_double_prime)))/(np.linalg.norm(r_prime)**3)
                    curvature_list.append(kappa)

                curvature_instance_array = np.array(curvature_list)

                # Now I will apply the state conditions
                """CALIBRATION PHASE"""
                if is_calibrating is True:
                    # collect data through the list quantity outside of the loop
                    calibration_curvature_array.append(curvature_instance_array)
                    if (time.time() - calibration_start_time) > calibration_duration:
                        # do the processing: simple test will just include the mean of the results but later ones will research into other methods
                        # like using a mix between the standard deviation, rolling mean
                        """
                        For future models, I will have the calibration phase be where the user moves to (a) a comfortable posititon, (b) defined positions that would be displayed (such as asking the user to tilt forward (there will be a diagam of what to do))
                        From this, I can develop an ML model (or just do more research into anatomical landmarks) and have a personalised "shape" for the user's spine; even if they have spinal deformities -- given the amount of input data from this calibration phase.
                        Then other forms of means can be applied (to be )
                        """
                        # mean value stored over this timestep (which doesn't actually need to be defined)
                        baseline_curvature = np.mean(calibration_curvature_array, axis=0)

                        # define the standard deviation of this distribution
                        curvature_standard_deviation = np.std(calibration_curvature_array, axis=0)

                        # leave this state
                        print("\nNOW EXITING CALIBRATION PHASE\n\n")
                        is_calibrating = False
                else:
                    # begin the monitoring mode
                    # compare the calibration code things, checking if the difference is within the standard deviation of each of the curvatures
                    if any((curvature_instance_array - baseline_curvature) > 1.5 * curvature_standard_deviation): # if there is a single instance of a deviation
                        """Poor posture has started. Start storing the indices of where the deviances have occurred and use this for later plotting."""
                        """The positions will also say where a signal should be sent along the spine"""

                        """
                        list structure that is 0 unless at the indicies where the baseline_curvature has exceeded
                        Then it will show by how much it has exceeded this level (the difference)
                        e.g. np.array([0,0,0,0,1,1.2,1.9,2.1,1.1,0.75,0.5,0,0,0,0,0])
                        It must retain the total length of the curvature_instance_array (as this index will be important)
                        """

                        print("THERE HAS BEEN DETECTED POOR POSTURE.")
                        deviation = (curvature_instance_array - baseline_curvature)
                        deviated_posture_instance = np.where((deviation > 1.5 * curvature_standard_deviation), deviation, 0)

                        poor_posture_list.append(deviated_posture_instance)

                        if poor_posture_start_time == None:
                            poor_posture_start_time = time.time()


                        elif (time.time() - poor_posture_start_time) > threshold_time:
                            """
                            There is now sustained poor posture, therefore model a distribution of the deviances of the posture. Depending on the threshold time, I could include a recency bias.
                            Send a signal to serial indicating the position of this poor posture and where the average of this is.
                            The signal processing (a dedicated function for ease) will send out:
                                1. The sensor(s) which is/are nearest to the deviations
                                2. The intensity of the haptic motor signal (from the distribution of peaks (should use deques for this aspect with the recency bias))
                                3. Tackle the issue of n-number deviances
                                4. Share this to the serial in this format: "serial.write(bf'Curve, {signal_response[0]}, {signal_response[1]}')"
                                5. Use the indices to plot the points in 3D space
                            """
                            poor_posture_list_numpy = np.array(poor_posture_list)
                            average_deviated_posture = np.array([interquartile_mean_calculation(poor_posture_list_numpy[:,i]) for i in range(len(poor_posture_list_numpy[0]))]) # i.e. for i in range({the length of each of the lists})

                            """
                            With this initial test, I will find the local maxima for the interpolated graph (using cubic interpolation; but will have this graph displayed for debugging)
                            """
                            deviated_graph_range = average_deviated_posture - baseline_curvature
                            """
                            # This may produce errors since the length of this list does not equate the thing that I will index it against
                            deviated_graph_domain = np.arange(len(deviated_graph_range))

                            # interpolate this into a function
                            deviated_graph_function = CubicSpline(deviated_graph_domain, deviated_graph_range)

                            # find the local maximum (and its index) for this

                            deviated_graph_domain_refined = np.linspace(0, len(deviated_graph_range), (len(deviated_graph_range) * 5)) # refined to approach the dx of finding the derivatives anyways
                            deviated_graph_range_refined = deviated_graph_function(deviated_graph_domain_refined)
                            """
                            peak_indices, dictionary_properties = scipy.signal.find_peaks(deviated_graph_range, prominence=None) # adjust the prominence values throughout through the test
                            # this can then be used to plot the position of the deviaiton (as a hotspot on the thing)
                            """
                            These indices are now the same for my hotspot which can be plotted as positions (which is fun and cool)
                            """
                            ## later, break down the fucntion into the points of the linear distances and get that all sorted out
                            """
                            Finding the linear distance thing is easy since I can just use the ratios between the distances and find the indices of them
                            Once I have done this bit I am free since I can print it out a red dot for where the deviation is and then be free
                            the place of the sensor can then be outputted from it
                            """
                            peak_incides_percentile = peak_indices / len(deviated_graph_range)
                            sensor_indices = []

                            for each_index in peak_incides_percentile:
                                # the argument where min(each_index - linear_distance_percentiles), i.e. the index in linear_distance_percentiles that this happens (because this is then the sensor that this happens in )
                                dist = np.abs(linear_distance_percentiles - each_index)
                                sensor_index = dist.argmin()
                                sensor_indices.append(sensor_index)

                            serialInst.write(f'DEVIATED_CURVE, {str(sensor_indices)}, [INTENSITY_VALUE]'.encode()) # there have been errors with the output of the sensor_indices not being a string

                            scatter_hotspot._offsets3d = (xc(plot_t[peak_indices]), yc(plot_t[peak_indices]), zc(plot_t[peak_indices]))
                            # the output to the serial will be a numpy array

                            return scatter, line, scatter_hotspot

                    else:
                        '''GOOD POSTURE'''
                        poor_posture_list = []
                        poor_posture_start_time = None
                        scatter_hotspot._offsets3d = ([], [], [])

                """PLOTTING LOGIC"""
                # Logic to allow the spline to be centred in frame (for easy visibility)
                centre_point = np.mean(IMU_POSITIONS, axis=0) # axis=0 is working along the column, axis=1 works along rows (especially with bigger matrices)

                ax.set_xlim(centre_point[0] - plot_radius, centre_point[0] + plot_radius)
                ax.set_ylim(centre_point[1] - plot_radius, centre_point[1] + plot_radius)
                ax.set_zlim(centre_point[2] - plot_radius, centre_point[2] + plot_radius)

                ## update stored variables ("scatter" + "line,") with new positions
                ## more efficient than ax.clear()

                scatter._offsets3d = (IMU_POSITIONS[:, 0], IMU_POSITIONS[:, 1], IMU_POSITIONS[:, 2]) # plots all x, y, z coordinates: regardless of number of rows in this matrix

                """
                Now plot the position
                """
                # line.set_data(IMU_POSITIONS[:, :2].T) # takes in (x,y) values of the points (matplotlib used to only do 2D stuff)
                # line.set_3d_properties(IMU_POSITIONS[:, 2]) # add the z-coordinate (3D space) on top of this

                line.set_data(xc(plot_t), yc(plot_t))
                line.set_3d_properties(zc(plot_t))
                ax.grid()

                ax2.clear()
                ax2.plot(plot_t, curvature_list)
                # print(max(curvature_list))
                # print(f"Current Avg: {np.mean(curvature_list)}")
                ax2.set_title("Curvature Graph")
                ax2.grid()

                """CALL THE .CSV FUNCTION/SQLITE DATA HERE TO STORE ALL THE DATA IN THE WAY THAT IS NECESSARY"""

        except (ValueError, IndexError) as e:
            print(f"Serial line error: {e}")
            # pass
            return scatter, line, scatter_hotspot

        return scatter, line, scatter_hotspot

    # PLOTTING DISPLAY
    anim = FuncAnimation(fig, animate, cache_frame_data=False, interval=100, blit=True) # blitting only draws the dynamic aspects of the plot
                                                                                        # rather than redrawing e.g. '{Title}' at each timestep
                                                                                        # this save computation
    ax.set_box_aspect([1,1,1])
    ax.set_proj_type('ortho')
    plt.tight_layout()
    plt.show()

### Exceptions
Although quite simple, this has helped to quickly identify bugs in the code and solve them.

In [ ]:
except Exception as e:
        exc_type, exc_value, exc_traceback = sys.exc_info()
            line_number = exc_traceback.tb_lineno
                print(f"ERROR: {e}, line {line_number}")